<a href="https://colab.research.google.com/github/teamasrmilano/Embeddings/blob/master/computing_sentence_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computing sentence embeddings

A first prototype on a small corpus, following https://openreview.net/pdf?id=SyK00v5xx.

The basic idea is to compute the sentence vector v_s as a weighted average of the word vectors associated with the words contained in s. The weight of each vector is an inverse function of the probability of the corresponding word in the corpus:

![s_formula](https://drive.google.com/uc?id=1rbDS6cuPPPm05V-6qEnUzX-PPcCYPP6h)

We need the following ingredients:

1) a set of sentences (corpus)

2) the vocabulary of corpus (unique tokens)

3) the probability of each word in the vocabulary (frequency in the corpus)

4) a word vector (embedding) for each word

Import basic packages first:

In [0]:
# basic
import numpy as np
import pandas as pd
import re # regular expressions

The parameter a is manually set to 0.001 (see paper):

In [0]:
a = 0.001

Next, we load our corpus, which is a random sample of 50k sentences from OpenSubtitles corpus available at http://opus.nlpl.eu/OpenSubtitles-v2018.php. The full corpus has been slightly cleaned and the sample obtained with bash command shuf.

In [0]:
with open('mini-subtitles-corpus', 'r') as input_file:
    corpus = input_file.read()

corpus = re.sub('\n', '. ', corpus) # we'll split sentences based on full stops with spacy

In [0]:
print(len(corpus))
corpus[0:1000]

2168174


"ok, beh, apprezzo la tua preoccupazione. avresti dovuto dirle prima queste cose. ti stai divertendo, blaine. sono miei ormai. quando ero a princeton, ho scritto la mia tesi sulle passioni degli stoici. hai una macchia sul tuo file, delinquente. presto anche tu capirai. se non lo fai, lo chiamo e gli dico dove sei. beh, sarebbe stato prima che venissi uccisa, ma funziona anche così. usa la presa al ginocchio. dolore e dai desideri. di che puntualizzazione parli. nella mia fantasia, noi ci trasferiamo nell'attico di jennifer lawrence. mazzola. perché loro non si lavano. non se ne adrà via. ah, 50ooo lire, don pietro. bob e malcolm sono stati licenziati. seq druven begnan si. molti amici mi hanno chiesto di farti ripensare a questa cosa. avrei dovuto lasciarla morire per evitare che lei odiasse me. erano un bersaglio facile mentre stavano dormendo nei loro nidi, di giorno. qual è stato il primo film western a vincere l'oscar come miglior film. magari mi fai vedere l'insegnante di inglese

A bunch of sentences, as expected.

Next, we parse the corpus with spacy (it'll take a while):

In [0]:
# import spaCy for nlp and italian resources (install if necessary)

#!pip3 install spacy
#!python3 -m spacy download it

import spacy
nlp = spacy.load('it')

In [0]:
nlp.max_length = 2500000
doc = nlp(corpus)

In [0]:
sentences = [sentence for sentence in doc.sents]

In [0]:
sentences[0:9]

[ok, beh, apprezzo la tua preoccupazione.,
 avresti dovuto dirle prima queste cose.,
 ti stai divertendo, blaine.,
 sono miei ormai.,
 quando ero a princeton, ho scritto la mia tesi sulle passioni degli stoici.,
 hai una macchia sul tuo file, delinquente.,
 presto anche tu capirai.,
 se non lo fai, lo chiamo e gli dico dove sei.,
 beh, sarebbe stato prima che venissi uccisa, ma funziona anche così.]

In [0]:
len(sentences)

49669

Get tokens, stripping punctuation:

In [0]:
tokens = [token.text for token in doc if token.is_punct != True]

In [0]:
len(tokens)

375400

Get frequency of each token:

In [0]:
# we use Counter from collections package
from collections import Counter

In [0]:
tokens_count = Counter(tokens)

For example:

In [0]:
tokens_count.most_common(10)

[('di', 9946),
 ('che', 9785),
 ('non', 8535),
 ('è', 8055),
 ('e', 6578),
 ('la', 6408),
 ('il', 6134),
 ('un', 5647),
 ('a', 5643),
 ('per', 4860)]

In [0]:
tokens_count['cane']

43

Get list of unique tokens:

In [0]:
unique_tokens = set(tokens)

In [0]:
vocab_size = len(unique_tokens)
print(vocab_size)

34870


With this list we can put together a dictionary of unique tokens with their probability in the corpus:

In [0]:
# iterating on the keys of tokens_count object, we divide the count of each token by the length of the vocabulary
tokens_prob = {key : tokens_count[key]/vocab_size for key in tokens_count.keys()}

For example:

In [0]:
tokens_prob['il']

0.17591052480642386

In [0]:
tokens_prob['cane']

0.0012331517063378262

In [0]:
tokens_prob['segugio']

2.8677946659019214e-05

Next, we train Word2Vec model on our corpus with gensim:

In [0]:
# gensim is used to load word embeddings (install if necessary)

#!pip3 install gensim

from gensim.models import Word2Vec

We need tokenized sentences as input for Word2Vec:

In [0]:
# double list comprehension: collect tokens, stripping punctuation, for each sentence in doc
tokenized_sentences = [[token.text for token in sentence if token.is_punct != True] for sentence in sentences]

For example:

In [0]:
tokenized_sentences[0:9]

[['ok', 'beh', 'apprezzo', 'la', 'tua', 'preoccupazione'],
 ['avresti', 'dovuto', 'dirle', 'prima', 'queste', 'cose'],
 ['ti', 'stai', 'divertendo', 'blaine'],
 ['sono', 'miei', 'ormai'],
 ['quando',
  'ero',
  'a',
  'princeton',
  'ho',
  'scritto',
  'la',
  'mia',
  'tesi',
  'sulle',
  'passioni',
  'degli',
  'stoici'],
 ['hai', 'una', 'macchia', 'sul', 'tuo', 'file', 'delinquente'],
 ['presto', 'anche', 'tu', 'capirai'],
 ['se', 'non', 'lo', 'fai', 'lo', 'chiamo', 'e', 'gli', 'dico', 'dove', 'sei'],
 ['beh',
  'sarebbe',
  'stato',
  'prima',
  'che',
  'venissi',
  'uccisa',
  'ma',
  'funziona',
  'anche',
  'così']]

In [0]:
vec_size = int(vocab_size ** 0.25) # rule of thumb to decide size of embedding vectors
model = Word2Vec(tokenized_sentences, size=vec_size, window=5, min_count=1, workers=4)

For example:

In [0]:
model.wv['cane'] # show only the first nine values

array([-0.01952695, -0.86842334, -0.22365926,  0.5114697 ,  0.14627382,
        0.19584416,  0.09706935, -0.3976206 , -0.7282566 ,  0.3901895 ,
       -0.8380402 , -0.11870573, -0.14886639], dtype=float32)

In [0]:
model.wv['lupo'][0:9] # show only the first nine values

array([-0.03319192, -0.32517394,  0.01581433,  0.24791561,  0.14155361,
        0.05693622,  0.08089875, -0.26927668, -0.36346108], dtype=float32)

In [0]:
model.wv['gatto'][0:9] # show only the first nine values

array([-0.02101748, -0.23991947, -0.01404044,  0.15411705,  0.11981014,
        0.10202154,  0.14525932, -0.23332986, -0.30248007], dtype=float32)

Double check vocabulary:

In [0]:
model_unique_tokens = set([token for token in model.wv.vocab]) # unique tokens in model vocab
model_unique_tokens == set(unique_tokens) # exaclty the same as unique_tokens above?

True

Cool.

We have all our ingredients: sentences, tokens, probabilities and vectors.

Let's move to sentence embedding algorithm:

In [0]:
def compute_s_vec(sentence, a=0.001): # make sure sentence is tokenized! 

    sent_vec = np.zeros(shape=vec_size) # initialize vector of zeros with the wanted shape

    for token in sentence: # cycle through tokens in sentence
        token_p = tokens_prob[token] # probability of token
        token_vec = model.wv[token] # token vector
        weighted_token_vec = token_vec*(a/(a+token_p)) # weighted vector of token
        sent_vec = sent_vec + weighted_token_vec # sum

    sent_vec = sent_vec*(1/len(sent_vec)) # average

    return(sent_vec)

For example:

In [0]:
il_cane_lupo = compute_s_vec(['il', 'cane', 'lupo'])
il_cane_lupo[0:9]

array([-0.00170747, -0.04825916, -0.0072154 ,  0.03150852,  0.01119856,
        0.01035368,  0.00720739, -0.02771687, -0.04392113])

In [0]:
il_cane_gatto = compute_s_vec(['il', 'cane', 'gatto'])
il_cane_gatto[0:9]

array([-0.00119312, -0.04501374, -0.00872881,  0.02751561,  0.01059617,
        0.01294983,  0.010912  , -0.02685651, -0.04210279])

Finally, create a dictionary computing vector for each sentence in corpus:

In [0]:
sent_vectors = {" ".join(tokenized_sentences[i]) : compute_s_vec(tokenized_sentences[i]) for i in range(len(sentences))}

Make a dataframe where columns are word vectors:

In [0]:
sent_vec_df = pd.DataFrame.from_dict(sent_vectors, orient='columns')

In [0]:
sent_vec_df.head()

,,1 0,1 00 miglia a est di tulip,1 4 aprile ore 2030,1 749 poco prima dell' orario di chiusura,14 placcato da una ragazza,14 settembre 1988,17 chiamami jeon jin ho,1x04 the pretender,3x08 all' the wisdom i got left,...,è vile e disgustosa,è vincolante,è viva e se la spassa a coral gables,è vivo ed è tornato vincitore,è volato via da uno dei piloni,è vuoto da parecchio tempo,è zane cannon,èmolto probabile che consiste di anidride carbonica,èrustico,èstato colpito alla testa
0,0.0,0.004612,0.013016,0.003639,0.009370,0.019924,0.006273,0.013148,0.008094,0.003100,...,0.001817,0.000004,0.020640,0.022562,0.005764,0.014449,-0.004111,0.010064,-0.002154,0.010237
1,0.0,-0.070560,-0.134504,-0.161405,-0.151702,-0.079950,-0.072541,-0.067501,-0.075093,-0.131364,...,-0.013355,0.001118,-0.040606,-0.100824,-0.082532,-0.072323,-0.002631,-0.046918,-0.001474,-0.082077
2,0.0,-0.020033,-0.030459,-0.035128,-0.037432,-0.017476,-0.014038,-0.014991,-0.020494,-0.035421,...,-0.005095,-0.002281,-0.006729,0.005096,-0.002382,-0.001981,-0.004906,-0.001525,0.001243,-0.020940
3,0.0,0.028402,0.062861,0.077565,0.066441,0.028200,0.030413,0.032750,0.042396,0.063013,...,0.001841,0.002872,0.016839,0.051775,0.022791,0.038079,-0.002188,0.015255,0.000192,0.020424
4,0.0,0.027282,0.047942,0.057292,0.052051,0.029038,0.030188,0.022540,0.027206,0.051764,...,0.001395,-0.000581,0.022016,0.029011,0.033898,0.020532,-0.003554,0.020382,-0.002646,0.021693


We can drop first two columns:

In [0]:
sent_vec_df = sent_vec_df.drop(columns = ['','1 0'])

In [0]:
sent_vec_df.head()

,1 00 miglia a est di tulip,1 4 aprile ore 2030,1 749 poco prima dell' orario di chiusura,14 placcato da una ragazza,14 settembre 1988,17 chiamami jeon jin ho,1x04 the pretender,3x08 all' the wisdom i got left,7 minuti,9 future tense,...,è vile e disgustosa,è vincolante,è viva e se la spassa a coral gables,è vivo ed è tornato vincitore,è volato via da uno dei piloni,è vuoto da parecchio tempo,è zane cannon,èmolto probabile che consiste di anidride carbonica,èrustico,èstato colpito alla testa
0,0.013016,0.003639,0.009370,0.019924,0.006273,0.013148,0.008094,0.003100,0.006417,0.007273,...,0.001817,0.000004,0.020640,0.022562,0.005764,0.014449,-0.004111,0.010064,-0.002154,0.010237
1,-0.134504,-0.161405,-0.151702,-0.079950,-0.072541,-0.067501,-0.075093,-0.131364,-0.072833,-0.046621,...,-0.013355,0.001118,-0.040606,-0.100824,-0.082532,-0.072323,-0.002631,-0.046918,-0.001474,-0.082077
2,-0.030459,-0.035128,-0.037432,-0.017476,-0.014038,-0.014991,-0.020494,-0.035421,-0.010374,-0.006055,...,-0.005095,-0.002281,-0.006729,0.005096,-0.002382,-0.001981,-0.004906,-0.001525,0.001243,-0.020940
3,0.062861,0.077565,0.066441,0.028200,0.030413,0.032750,0.042396,0.063013,0.037878,0.026234,...,0.001841,0.002872,0.016839,0.051775,0.022791,0.038079,-0.002188,0.015255,0.000192,0.020424
4,0.047942,0.057292,0.052051,0.029038,0.030188,0.022540,0.027206,0.051764,0.029619,0.021295,...,0.001395,-0.000581,0.022016,0.029011,0.033898,0.020532,-0.003554,0.020382,-0.002646,0.021693


In [0]:
#sent_vec_df.to_csv("sentence_vectors.tsv", sep='\t', index=False, index_label=False)

As matrix:

In [0]:
sent_vec_matrix = sent_vec_df.to_numpy() 

In [0]:
sent_vec_matrix.shape

(13, 49664)

In [0]:
sent_vec_matrix[:,0:9]

array([[ 0.01301633,  0.0036387 ,  0.00936968,  0.01992386,  0.00627333,
         0.01314799,  0.00809409,  0.00309993,  0.0064167 ],
       [-0.13450362, -0.16140485, -0.15170162, -0.07994987, -0.07254116,
        -0.0675015 , -0.07509253, -0.13136393, -0.07283299],
       [-0.03045868, -0.035128  , -0.03743216, -0.01747611, -0.01403762,
        -0.01499107, -0.02049371, -0.03542052, -0.01037356],
       [ 0.06286137,  0.07756484,  0.06644113,  0.02820012,  0.0304128 ,
         0.03275006,  0.04239554,  0.06301266,  0.03787784],
       [ 0.04794178,  0.05729186,  0.05205051,  0.02903815,  0.03018784,
         0.02254012,  0.02720561,  0.05176378,  0.02961893],
       [ 0.03973577,  0.05337564,  0.05080131,  0.02001939,  0.0189963 ,
         0.0219002 ,  0.02693642,  0.04425231,  0.02712512],
       [-0.02029607, -0.02738428, -0.01452657,  0.00192814, -0.00982968,
        -0.00623123, -0.01644851, -0.03353185, -0.01154771],
       [-0.01418074, -0.01596084, -0.02179708, -0.01198874, -0

Extract first singular vector of the matrix, using svd from scikit-learn:

In [0]:
from sklearn.utils.extmath import randomized_svd

In [0]:
U, S, vt = randomized_svd(sent_vec_matrix, n_components=1)

In [0]:
U

array([[-0.14388128],
       [ 0.53930292],
       [ 0.10229087],
       [-0.21976348],
       [-0.21930481],
       [-0.16118187],
       [ 0.00646664],
       [ 0.16959303],
       [ 0.49321712],
       [-0.20826905],
       [ 0.47713661],
       [ 0.04614985],
       [ 0.10190706]])

In [0]:
From U to UUT:

In [0]:
UUT = np.outer(U,U.T)
UUT.shape

(13, 13)

Let's look at one example:

In [0]:
sent_vec_df['7 minuti']

0     0.006417
1    -0.072833
2    -0.010374
3     0.037878
4     0.029619
5     0.027125
6    -0.011548
7    -0.007647
8    -0.058860
9     0.024990
10   -0.056865
11   -0.003414
12   -0.013347
Name: 7 minuti, dtype: float64

In [0]:
np.dot(UUT, sent_vec_df['7 minuti'])

array([ 0.01794381, -0.06725786, -0.01275696,  0.02740727,  0.02735007,
        0.02010141, -0.00080647, -0.02115038, -0.06151038,  0.02597377,
       -0.05950494, -0.00575547, -0.01270909])

In [0]:
sent_vec_df['7 minuti'] - np.dot(UUT, sent_vec_df['7 minuti'])

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:19: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.6.1

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


0    -0.011527
1    -0.005575
2     0.002383
3     0.010471
4     0.002269
5     0.007024
6    -0.010741
7     0.013503
8     0.002650
9    -0.000984
10    0.002640
11    0.002342
12   -0.000638
Name: 7 minuti, dtype: float64

Apply this to whole dataset:

In [0]:
for i in range(sent_vec_df.shape[1]):
    sent_vec_df.iloc[:,i] = sent_vec_df.iloc[:,i] - np.dot(UUT, sent_vec_df.iloc[:,i])

In [0]:
sent_vec_df.iloc[:,0:9]

NameError: ignored

Save to disk:

In [0]:
sent_vec_df.to_csv("sentence_embeddings.tsv", sep="\t", index=False, index_label=False)

NameError: ignored